# Data Vis: Client-Side Interactivity
* Notebook 2: Brushing with Plotly

## Setup

In [2]:
import pandas as pd
import plotly.express as px
import plotly.graph_objects as go
from ipywidgets import VBox, HBox, Button
from IPython.display import display

## Data

In [3]:
data = pd.read_csv("flights_joined.csv")
data = data.sample(1000, random_state=42)

In [4]:
data.head()

,year,month,day,dep_time,sched_dep_time,dep_delay,arr_time,sched_arr_time,arr_delay,carrier,...,tzone_dest,temp,dewp,humid,wind_dir,wind_speed,wind_gust,precip,pressure,visib
159280,2013,3,25,1929.0,1905,24.0,2236.0,2217,19.0,UA,...,America/New_York,37.94,35.06,89.25,320.0,9.20624,NaN,0.02,999.8,6.0
189101,2013,4,26,956.0,1000,-4.0,1257.0,1334,-37.0,DL,...,America/Los_Angeles,60.08,33.08,36.04,330.0,8.05546,16.11092,0.00,1029.2,10.0
212435,2013,5,21,1320.0,1309,11.0,1430.0,1414,16.0,EV,...,America/New_York,86.00,68.00,55.04,240.0,11.50780,NaN,0.00,1012.2,10.0
266804,2013,7,18,1222.0,1230,-8.0,1357.0,1419,-22.0,EV,...,America/New_York,98.06,69.08,39.22,300.0,9.20624,17.26170,0.00,1016.7,10.0
306581,2013,8,29,540.0,545,-5.0,921.0,921,0.0,B6,...,NaN,71.96,68.00,87.35,100.0,8.05546,NaN,0.00,1014.0,10.0


## Brushing

So far, we have been using Plotly to create single interactive visualizations. However, Plotly also supports between-plot interactivity, which allows us to select points in one plot and have those selections reflected in another plot. This is known as "brushing".

In the code below, we use `plotly.graph_objects` and `FigureWidget` to create a bar chart and a scatter plot. When we click on a bar in the bar chart, the scatter plot updates by changing the color of the points associated with the selected bar.

In [ ]:
# 1) Define categories for the bar chart and keep track of which categories are “selected”
cats = data["origin"].unique().tolist()
selected_cats = set()

# 2) Create bar chart using Plotly FigureWidget
bar_fig = go.FigureWidget(
    go.Bar(x=cats, y=data.groupby("origin").size().reindex(cats).tolist(), 
        marker_color=["steelblue"]*len(cats))
)
bar_fig.update_layout(title="Click bars to (de)select",
                      xaxis_title="Origin",
                      width=400, height=350)

# 3) Create scatter plot using Plotly FigureWidget
scatter_fig = go.FigureWidget(
    go.Scatter(x=data["arr_delay"], y=data["dep_delay"],
            mode="markers", marker=dict(size=10, color="lightgray"))
)
scatter_fig.update_layout(title="Scatter (highlights follow bar clicks)",
                          xaxis_title="Arrival Delay",
                          yaxis_title="Departure Delay",
                          width=400, height=350)

# 4) Create handler to toggle the categories in the set and update the colors
def on_bar_click(trace, points, selector):

    # a) Toggle the selected categories based on the clicked bar
    for i in points.point_inds:
        cat = cats[i]
        if cat in selected_cats:
            selected_cats.remove(cat)
        else:
            selected_cats.add(cat)

    # b) Update bar colors
    new_bar_colors = []
    for c in cats:
        if c in selected_cats:
            new_bar_colors.append("crimson")
        else:
            new_bar_colors.append("steelblue")
    bar_fig.data[0].marker.color = new_bar_colors

    # c) Update scatter colors
    new_scatter_colors = []
    for c in data["origin"]:
        if c in selected_cats:
            new_scatter_colors.append("crimson")
        else:
            new_scatter_colors.append("lightgray")
    scatter_fig.data[0].marker.color = new_scatter_colors

# 5) Register the click handler
bar_fig.data[0].on_click(on_bar_click)

# 6) Arrange everything (1 button and 2 plots) in a layout
display(HBox([bar_fig, scatter_fig]))


    'data': [{'marker': {'color': ['steelblue', 'steelblue', 'steelblue']},
    …

## Your Turn (20 minutes)

Create a vertical layout with two plots: a scatter plot in the first row and a bar chart in the second row. The scatter plot should show the relationship between hour of the day and departure delay. The bar chart should show the number of flights per airline. When the user clicks on a bar, the scatter plot should update to show only the points associated with the selected airline.